In [ ]:
session = get_active_session()
df = session.sql("SELECT * FROM MEDICARE.PUBLIC.MEDICARE_TABLE LIMIT 1000")
df.collect()
pandas_df = df.to_pandas()

In [ ]:

from io import BytesIO
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import joblib
import json
from azure.storage.blob import BlobServiceClient
from snowflake.snowpark.files import SnowflakeFile
import snowflake.connector
from snowflake.snowpark.context import get_active_session

def preprocess_data(input_data):

    # Initialize Snowflake session

    session = get_active_session()
    # Load and preprocess data
    model_file = "@MEDICARE.PUBLIC.DATA/dataset1.csv"
    #df = pd.read_csv(SnowflakeFile.open(model_file, 'rb', require_scoped_url=False))
    session = get_active_session()
    df = session.sql("SELECT * FROM MEDICARE.PUBLIC.MEDICARE_TABLE LIMIT 1000")
    df.collect()
    df =  df.to_pandas()

    X = df.drop('V28HCCCODED', axis=1)
    y = df['V28HCCCODED']
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Apply transformations
    X_transformed = preprocessor.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_transformed, y, test_size=0.2, random_state=42, stratify=y)
    print('Hii')

    # Train model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    print('model trained')

    # Save model to in-memory bytes buffer
    local_file_path = './logistic_regression_model.pkl'
    joblib.dump(model, local_file_path)
    buffer = BytesIO()
    joblib.dump(model, buffer)
    buffer.seek(0)
    stage_name = '@MEDICARE.PUBLIC.DATA'
    put_result = session.file.put(local_file_path, "@MEDICARE.PUBLIC.DATA/logistic_regression_model.pkl")
    #FileOperation.put(local_file_name = local_file_path,stage_location = stage_name)

   

    return put_result[0]




In [ ]:
preprocess_data('Hi')